In [1]:
import pandas as pd

df = pd.read_csv("./mental-heath-in-tech-2016_20161114.csv")

pd.set_option('max_colwidth', 300)
pd.set_option('max_columns', 70)

In [ ]:
# Columns still with null (start from 0): 
# 1 (company size), 2 (primarily tech?), 3 (role in company)
# 10/11 (discussing mental health/physical negative consequences?)
# 12 (comfortable discussing a mental health disorder with your coworkers)
# 13 (comfortable discussing a mental health disorder with your direct supervisor(s)?)
# 15 (observed negative consequences for co-workers who have been open about mental health issues in your workplace?
# 16 (medical coverage which includes treatment of  mental health issues?
# 17 (know online/local resources)
# 18 (If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?)
# 19: If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?

In [27]:
# Clean Data

# Handle NaN

# 4: Does your employer provide mental health benefits as part of healthcare coverage?
# 6: Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?
# 7: Does your employer offer resources to learn more about mental health concerns and options for seeking help?
# 8: Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?
# 9: If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:
# 14: Do you feel that your employer takes mental health as seriously as physical health?
df.iloc[:, [4, 6, 7, 8, 9, 14]] = df.iloc[:, [4, 6, 7, 8, 9, 14]].fillna("I don't know")
# 5: Do you know the options for mental health care available under your employer-provided coverage?
df.iloc[:, 5].fillna("I am not sure", inplace=True)




male = ['Male', 'male', 'Male ', 'M', 'm', 'man', 'Cis male', 'Male.', 'Male. ', 'Male (cis)', 'Man', 'Sex is male',
        'cis male', 'Malr', 'Dude', 'mail', 'M|', 'male ', 'Cis Male', 'cisdude', 'cis man', 'MALE',
        "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? "]
female = ['Female', 'female', 'I identify as female.', 'female ', 'Female assigned at birth ', 'F', 'Woman', 'f',
          'Cis female ', 'Female ', 'woman', 'female/woman', 'Cisgender Female', 'fem', 'Female (props for making this a freeform field, though)',
          ' Female', 'Cis-woman']
other = ['Bigender', 'non-binary', 'fm', 'Transitioned, M2F', 'Genderfluid (born female)', 'Other/Transfeminine',
         'Female or Multi-Gender Femme', 'Androgynous', 'male 9:1 female, roughly', 'Other', 'nb masculine',
         'none of your business', 'genderqueer', 'Human','Genderfluid', 'Enby','genderqueer woman', 'mtf', 'Queer',
         'Agender', 'Fluid', 'Male/genderqueer', 'Nonbinary', 'human', 'Unicorn', 'Male (trans, FtM)',
         'Genderqueer', 'Genderflux demi-girl', 'female-bodied; no feelings about gender', 'AFAB', 'Transgender woman']

df['What is your gender?'] = df['What is your gender?'].replace(male, 'male')
df['What is your gender?'] = df['What is your gender?'].replace(female, 'female')
df['What is your gender?'] = df['What is your gender?'].replace(other, 'other')

df['What is your gender?'].unique()

array(['male', 'female', 'other', nan], dtype=object)

In [28]:
df.iloc[:,19]

0       I'm not sure
1       I'm not sure
2       I'm not sure
3       I'm not sure
4       I'm not sure
            ...     
1428    I'm not sure
1429              No
1430    I'm not sure
1431    I'm not sure
1432    I'm not sure
Name: If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?, Length: 1433, dtype: object